In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

# 3 observations of a single parameter (x) and their corresponding output (y)
X = np.array([[1], [3], [5]])  # 1D input data (shape is (n_samples, 1))
y = np.array([2, 3, 5])  # Observations (outputs)

# Define the kernel for the GP: constant kernel times RBF kernel
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))

# Create a Gaussian Process model
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Fit the model to the data
gp.fit(X, y)

# Generate test data for prediction (over a grid of points for visualization)
X_test = np.linspace(0, 6, 100).reshape(-1, 1)  # Input space for predictions

# Predict using the GP model
y_pred, sigma = gp.predict(X_test, return_std=True)  # Get both mean and uncertainty (std)

# Define Expected Improvement (EI) acquisition function
def expected_improvement(X, X_sample, Y_sample, gp, xi=0.01):
    """
    Computes the EI at points X based on existing samples X_sample and Y_sample
    from the Gaussian process model.

    Arguments:
    X -- Points at which EI shall be computed (test points)
    X_sample -- Sampled input points (observations)
    Y_sample -- Corresponding observed function values
    gp -- A fitted GaussianProcessRegressor model
    xi -- Exploitation-exploration trade-off parameter

    Returns:
    EI values for the test points X.
    """
    mu, sigma = gp.predict(X, return_std=True)
    
    # Ensure sigma is reshaped to avoid dimension mismatch
    sigma = sigma.reshape(-1, 1)
    
    # Best observed value so far (exploitation target)
    mu_sample_opt = np.max(Y_sample)

    # Compute the improvement
    imp = mu - mu_sample_opt - xi
    
    # Normalize the improvement by the uncertainty
    Z = np.divide(imp, sigma, out=np.zeros_like(imp), where=sigma!=0)

    # Expected Improvement calculation using the CDF and PDF of normal distribution
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    
    # EI should be 0 where sigma is 0 to prevent NaN values
    ei[sigma == 0.0] = 0.0
    
    return ei

# Calculate the Expected Improvement (EI) at each test point
ei = expected_improvement(X_test, X, y, gp)

# Plot the results
plt.figure(figsize=(10, 6))

# Plot the GP mean prediction
plt.plot(X_test, y_pred, 'b-', label='GP mean')

# Plot the uncertainty (confidence interval)
plt.fill_between(X_test.ravel(), 
                 y_pred - 1.96 * sigma, 
                 y_pred + 1.96 * sigma, 
                 alpha=0.2, color='blue', label='95% confidence interval')

# Plot the original observations
plt.scatter(X, y, c='r', s=100, zorder=10, label='Observations')

# Plot the acquisition function (scaled for visualization)
plt.plot(X_test, ei, 'g--', label='Acquisition (EI)')

plt.title('Gaussian Process Regression with Acquisition Function')
plt.xlabel('Parameter')
plt.ylabel('Observation / Acquisition Value')
plt.legend()
plt.show()

ValueError: non-broadcastable output operand with shape (100,) doesn't match the broadcast shape (100,100)